# Descripción

Este notebook contiene los pasos requeridos para asignar las combinaciones, previamente asignadas, profesor-curso a un salón de acuerdo a los requerimientos institucionales.

In [14]:
import json


# READ DATA

## first stage model output

In [15]:
# basic parameters
DAY_LENGTH=16

# Horizonte de planificación
HORIZON=5

#  Días de la semana
DAYS=['Lunes','Martes','Miercoles','Jueves','Viernes']


In [16]:
# Este diccionario contiene un mapa de los slots de tiempo a los días y horas

actual_time={}
for d in range(HORIZON):
    for h in range(DAY_LENGTH):
        actual_time[d*16+h]={"slot":DAYS[d]+' '+str(h+6)+':00', "day":d, "hour":h,'slot_abr':DAYS[d][0]+str(h)}

In [17]:
# read model_output.json
allocations=json.load(open('data/model_output.json','r'))
#allocations

In [18]:

groups=allocations['courses']
teachers=allocations['teachers']
pairs=allocations['ensembles']
teachers=set([int(t) for t in teachers])
groups=set([g for g in groups])
pairs=[(int(p[0]) ,p[1]) for p in pairs]


In [19]:
courses_by_teacher={}
for p in pairs:
    if p[0] not in courses_by_teacher:
        courses_by_teacher[p[0]]=[p[1]]
    else:
        courses_by_teacher[p[0]].append(p[1])


## Initial data load

In [20]:
# read data.json
horas_curso=json.load(open('data/horas_curso.json','r'))
grupo_requerimiento=json.load(open('data/grupo_requerimiento.json','r'))
capacidad_salones=json.load(open('data/capacidad_sala.json','r'))
ubicacion_semestral=json.load(open('data/ubicacion_semestral.json','r'))

# Model construction

In [21]:
I=set(teachers)
J=set(groups)
K=set(pairs)
C=set(capacidad_salones.keys())


from itertools import product

# courses that are the same
same_course={}
# courses that are taught by the same teacher
same_teacher={}

# delta semestres
delta_semestre={}
for j1,j2 in product(J,J):
    same_course[(j1,j2)]=0
    same_teacher[(j1,j2)]=0
    if j1[:9]==j2[:9]:
        same_course[(j1,j2)]=1
    delta_semestre[(j1,j2)]=abs(ubicacion_semestral[j1[:9]]["Semestre"]-ubicacion_semestral[j2[:9]]["Semestre"])

for p in pairs:
    for p2 in pairs:
        if p[0]==p2[0]:
            same_teacher[(p[1],p2[1])]=1

# course and time slot compatibility
time_compatibility={}
for c,j in product(C,J):
    for s in actual_time.keys():
        time_compatibility[(c,j,s)]=0
        if actual_time[s]['hour'] + horas_curso[j[:9]]['horas']<=DAY_LENGTH:
            time_compatibility[(c,j,s)]=1

valid_keys=[]
for c,j,s in product(C,J,actual_time.keys()):
    if time_compatibility[(c,j,s)]==1:
        valid_keys.append((c,j,s))

In [22]:
import gurobipy as gp

# declare an empty model
m=gp.Model('Asignacion de salones')
m.setParam('OutputFlag',False)

# model sense
m.modelSense=gp.GRB.MAXIMIZE

# declares variables

y=m.addVars(valid_keys,vtype=gp.GRB.BINARY,name='y')
x=m.addVars([(c,j,s) for c in C for j in J for s in actual_time.keys()],vtype=gp.GRB.BINARY,name='x')
z=m.addVars([(j1,j2) for j1,j2 in product(J,J) if j1!=j2],vtype=gp.GRB.BINARY,name='z')


$ \max:\sum_{{j1,j2 \in J, j1 \neq j2}} z[j1,j2] \cdot \delta_{\text{{semestre}}}[j1,j2]$


In [23]:
# funcion objetivo:
# maximizar la suma de los deltas semestrales de los cursos que se imparten
m.setObjective(gp.quicksum(z[j1,j2]*delta_semestre[j1,j2] for j1,j2 in product(J,J) if j1!=j2))

$\sum_{c,j1,s \in \text{{valid\_keys}}, j1=j} y[(c,j1,s)] = 1 \quad \forall j \in J$



In [25]:
# each course starts at one time slot
m.addConstrs((gp.quicksum(y[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==1 for j in J),name='each_course_starts_at_one_time_slot')
0

0

$  
\sum_{s1 \in \text{{range}}(s,s+\text{{horas\_curso}}[j1[:9]]['horas']) \atop (c,j1,s1) \in \text{{valid\_keys}}} x[(c,j1,s1)] \geq y[(c,j1,s)] \cdot \text{{horas\_curso}}[j1[:9]]['horas'] \forall c, j1, s \in \text{{valid\_keys}}
$


In [ ]:
# if a course start at time slot s in classroom c, it should last for the required time

m.addConstrs((gp.quicksum(x[(c,j1,s1)] for s1 in range(s,s+horas_curso[j1[:9]]['horas'])  if (c,j1,s1) in valid_keys)>=y[(c,j1,s)]*horas_curso[j1[:9]]['horas'] for c,j1,s in valid_keys),name='if_a_course_start_at_time_slot_s_in_classroom_c_it_should_last_for_the_required_time')
print("Hello world")

In [26]:
# each course is assigned the time it requires
m.addConstrs((gp.quicksum(y[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==horas_curso[j[:9]]['horas'] for j in J),name='each_course_is_assigned_the_time_it_requires')
0

{'8827-0008-0': <gurobi.Constr *Awaiting Model Update*>,
 '8828-0028-0': <gurobi.Constr *Awaiting Model Update*>,
 '8830-0004-0': <gurobi.Constr *Awaiting Model Update*>,
 '6069-0175-0': <gurobi.Constr *Awaiting Model Update*>,
 '8830-0009-0': <gurobi.Constr *Awaiting Model Update*>,
 '9937-0021-0': <gurobi.Constr *Awaiting Model Update*>,
 '6612-0001-0': <gurobi.Constr *Awaiting Model Update*>,
 '6611-0004-0': <gurobi.Constr *Awaiting Model Update*>,
 '6611-0088-0': <gurobi.Constr *Awaiting Model Update*>,
 '8830-0001-0': <gurobi.Constr *Awaiting Model Update*>,
 '6069-0154-0': <gurobi.Constr *Awaiting Model Update*>,
 '5555-0010-0': <gurobi.Constr *Awaiting Model Update*>,
 '8830-0011-0': <gurobi.Constr *Awaiting Model Update*>,
 '8830-0007-0': <gurobi.Constr *Awaiting Model Update*>,
 '6069-0127-0': <gurobi.Constr *Awaiting Model Update*>,
 '8831-0028-0': <gurobi.Constr *Awaiting Model Update*>,
 '9939-0018-0': <gurobi.Constr *Awaiting Model Update*>,
 '9937-0062-0': <gurobi.Constr 

In [27]:
# si dos cursos se imparten a la misma hora en cualquier salon se debe activar la variable z, que lleva cuentas de los solapamientos
m.addConstrs((z[j1,j2]>=x[c1,j1,s]+x[c2,j2,s]-1 for j1,j2 in product(J,J) if j1!=j2 for c1 ,c2 in product(C,C) for s in actual_time.keys()),name='si_dos_cursos_se_imparten_a_la_misma_hora_en_cualquier_salon_se_debe_activar_la_variable_z_que_lleva_cuentas_de_los_solapamientos')
0

In [48]:
# si dos cursos se imparten a la misma hora en cualquier salon se debe activar la variable z, que lleva cuentas de los solapamientos

for j1,j2 in product(J,J):
    if j1!=j2:
        for c1 ,c2 in product(C,C):
            for s in actual_time.keys():
                model+=z[(j1,j2)]>=x[(c1,j1,s)]+x[(c2,j2,s)]-1
                

KeyboardInterrupt: 

In [ ]:
time_limit=3600
m.setParam('TimeLimit',time_limit)
m.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/Library/Caches/pypoetry/virtualenvs/timetabling_upb/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/2d0a6ae1821d420486a377988456e12e-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/2d0a6ae1821d420486a377988456e12e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 121 COLUMNS
At line 727340 RHS
At line 727457 BOUNDS
At line 969864 ENDATA
Problem MODEL has 116 rows, 242406 columns and 239100 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.22 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was

1